In [ ]:
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
import json

# Cargar el modelo preentrenado XGBoost para predicción del salario
xgboost_model = joblib.load('xgboost_model.pkl')

# Cargar las etiquetas desde el archivo JSON
with open('labels.json', 'r') as f:
    labels = json.load(f)

# Cargar el modelo Zero-Shot de Facebook BART para clasificación de etiquetas
device = -1  # Usamos CPU en transformers
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# Función para extraer la descripción del trabajo desde la URL usando Selenium
def extract_job_description(url):
    options = webdriver.ChromeOptions()
    options.headless = True  # Ejecutar en modo sin cabeza (sin ventana de navegador)
    
    # Iniciar el navegador (Selenium)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Abrir la URL
    driver.get(url)
    
    # Esperar un poco para asegurarse de que el contenido esté cargado
    time.sleep(2)  # Puede ajustarse según el tiempo de carga de la página

    # Intentar encontrar el botón "Ver más" y hacer clic en él
    try:
        ver_mas_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Ver más')]")
        ver_mas_button.click()
        time.sleep(2)
    except Exception as e:
        print(f"Error al intentar hacer clic en el botón 'Ver más': {e}")

    # Extraer el HTML de la página después de que el contenido haya sido desplegado
    page_source = driver.page_source

    # Cerrar el navegador
    driver.quit()

    # Usar BeautifulSoup para analizar el contenido extraído
    soup = BeautifulSoup(page_source, 'html.parser')

    # Buscar el div que contiene la clase 'jobs-box__html-content'
    job_content = soup.find('div', class_='jobs-box__html-content')
    
    if job_content:
        # Extraer el texto de los elementos <p> y <li>
        description_text = "\n".join([p.get_text() for p in job_content.find_all('p')])
        description_text += "\n".join([li.get_text() for li in job_content.find_all('li')])
        return description_text.strip()
    return "Descripción no encontrada"

# Función para clasificar la descripción con el modelo Zero-Shot
def classify_description(description):
    classification_results = {}
    for category, prompts in labels.items():
        output = classifier(description, candidate_labels=prompts)
        classification_results[category] = output['scores'][0]  # Mejor predicción
    return classification_results

# Función para predecir el salario usando el modelo XGBoost
def predict_salary(description):
    # Preprocesar la descripción con TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([description])
    
    # Realizar la predicción del salario
    predicted_salary = xgboost_model.predict(tfidf_matrix)
    
    # Retornar el salario predicho
    return predicted_salary[0]

# Interfaz de Streamlit
def main():
    st.title("Clasificación y Predicción del Salario de Oferta de Trabajo")
    url = st.text_input("Introduce la URL de la oferta de trabajo:")
    
    if url:
        with st.spinner('Extrayendo datos...'):
            description = extract_job_description(url)
            if description == "Descripción no encontrada":
                st.error("No se pudo encontrar la descripción en la URL proporcionada.")
            else:
                st.write(f"Descripción extraída:\n{description}")
                
                # Clasificar la oferta de trabajo
                st.write("Clasificando la oferta de trabajo...")
                classification_results = classify_description(description)
                
                st.write("Resultados de clasificación:")
                for category, score in classification_results.items():
                    st.write(f"{category}: {score:.2f}")
                
                # Predecir el salario de la oferta de trabajo
                st.write("Prediciendo el salario...")
                predicted_salary = predict_salary(description)
                st.write(f"Salario Predicho: ${predicted_salary:.2f}")

# Ejecutar la aplicación
if __name__ == "__main__":
    main()


In [ ]:
extract_job_description("https://www.linkedin.com/jobs/view/4225618871/?alternateChannel=search&refId=10mbEPkt11Kh5p%2B%2F%2BnvG%2Bg%3D%3D&trackingId=D9m5tiY%2FIVxCvl2biaDwHg%3D%3D")